## Get secondary structure from alphafold PDB files

AF = alphafold

This script converts uniprotIds into alphafold urls, downloads these pdb files, runs dssp on these files, and converts them into dataframes storing secondary structure info saved as .pkl objects

Add url for download

In [ ]:
import pandas as pd

In [8]:
os.chdir('..')
os.getcwd()

'/cluster/gjb_lab/2472402'

In [ ]:
df = pd.read_csv('1497_uniprotId.csv')

In [ ]:
for i,row in df.iterrows():
    isna = pd.Series(row.uniprotId).isna()[0]
    if not isna and re.compile('^[A-Z](\d+)').search(row.uniprotId):
        url = 'https://alphafold.ebi.ac.uk/files/AF-%s-F1-model_v1.pdb' % row.uniprotId
        df.loc[i,'url'] = url
    else:
        url=None

In [ ]:
df

,seqID,domain,pdbId,chain,entity,uniprotId,sequence,url
0,24695,d1a12a_,1a12,a,1.0,P18754,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...,https://alphafold.ebi.ac.uk/files/AF-P18754-F1...
1,24696,d1a1xa_,1a1x,a,1.0,P56278,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,https://alphafold.ebi.ac.uk/files/AF-P56278-F1...
2,24697,d1a62a1,1a62,a,1.0,P0AG30,*NLTELKNTPVSELITLGEN*GLENLAR*RKQDIIFAILKQHAKSG...,https://alphafold.ebi.ac.uk/files/AF-P0AG30-F1...
3,24698,d1a9xa1,1a9x,a,1.0,PRO_0000145004,MPKRTDIKSILILGAGPIVIGQACEFDYSGAQACKALREEGYRVIN...,NaN
4,24699,d1a9xb1,1a9x,b,1.0,PRO_0000145004,MPKRTDIKSILILGAGPIVIGQACEFDYSGAQACKALREEGYRVIN...,NaN
...,...,...,...,...,...,...,...,...
1492,26187,d4lrsa2,4lrs,a,1.0,D1A3K8,MTDQDGATAAPRVRITDSTLRDGSHAMAHQFTEEQVRATVHALDAA...,https://alphafold.ebi.ac.uk/files/AF-D1A3K8-F1...
1493,26188,d4lwra_,4lwr,a,NaN,NaN,NaN,NaN
1494,26189,d4mf5a2,4mf5,a,1.0,B1FXZ2,MHHHHHHSSGVDLGTENLYFQSMTDLSAFPITKKWPAAHPERLQLY...,https://alphafold.ebi.ac.uk/files/AF-B1FXZ2-F1...
1495,26190,d4mjra1,4mjr,a,1.0,P0A988,MKFTVEREHLLKPLQQVSGPLGGRPTLPILGNLLLQVADGTLSLTG...,https://alphafold.ebi.ac.uk/files/AF-P0A988-F1...


Save as the same df but with additional column

In [94]:
df.to_csv('1497_url.csv',index=False)

Running the cells below creates 1343 pdb files in AF-pdb-files-jnet directory

In [111]:
import pandas as pd
import os
import logging
import traceback
import requests

A bit of test code

In [48]:
df = pd.read_csv('1497_uniprotId.csv')

In [107]:
testurl=df.url[0]
r = requests.get(testurl)
with open('helloworld.pdb','w+') as f:
    f.write(r.text)

Actual loop to download pdb files

In [115]:
for i,row in df.iterrows():
    url_exists = not pd.Series(row.url).isna()[0]
    if url_exists:
        filename = '/cluster/gjb_lab/2472402/AF-pdb-files-jnet/%d.pdb' % row.seqID
        try:
            r = requests.get(row.url)
            with open(filename,'w+') as f:
                f.write(r.text)
        except Exception as e:
            logging.log(traceback(e))

Covert pdb files into dssp files using pdb_to_dssp.py script

>Run this in terminal

>```cd /cluster/gjb_lab/2472402```

>```qsub scripts/pdb_to_dssp_batch.py AF-pdb-files-jnet```

Convert dssp output into pandas dataframes and save as pkl

>Run this in terminal

>Writes.pkl files in same directory as .dssp files

> ```scripts/parse_dssp_to_ss.py AF-pdb-files-jnet AF-pdb-files-jnet```

Visualize alphafold SS

In [23]:
import glob
import os
import pickle
import pandas as pd

In [5]:
os.chdir('..')

In [8]:
import pickle
import glob

In [9]:
pkl_file_list = glob.glob('AF-pdb-files-jnet/*.pkl')

In [10]:
pkl_df_list = []

for pkl_file in pkl_file_list:
    with open(pkl_file,'rb') as f:
        ss_df = pickle.load(f)
        pkl_df_list.append(ss_df)

In [18]:
%whos

Variable        Type              Data/Info
-------------------------------------------
f               BufferedReader    <_io.BufferedReader name=<...>db-files-jnet/25810.pkl'>
glob            module            <module 'glob' from '/usr<...>lib64/python3.6/glob.py'>
os              module            <module 'os' from '/usr/lib64/python3.6/os.py'>
pickle          module            <module 'pickle' from '/u<...>b64/python3.6/pickle.py'>
pkl_file        str               AF-pdb-files-jnet/25810.pkl
pkl_file_list   list              n=451
pkl_obj_list    list              n=451
seqID_list      list              n=451
ss_df           DataFrame              site amino_acid  ASA<...>n\n[326 rows x 6 columns]
test_df         DataFrame              site amino_acid  ASA<...>n\n[538 rows x 6 columns]


In [12]:
test_df = pkl_df_list[0]

In [14]:
''.join(test_df.SS)

'-----SSHHHHTTSSSS---HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHTT---SS----GGGSTT----TT-SS-HHHHHHHHSS-HHHHHHHHHHHHHHHT-SS---HHHHHHSTTHHHHHHHHTT-TT-HHHHHHHHHHHHHHHTS-HHHHHHHHHTTHHHHHHHHTT-S-HHHHHHHHHHHHHHHTT-HHHHHHHHHTT-HHHHHHHHHH---HHHHHHHHHHHHHHH--SSS---HHHHGGGHHHHHHHTT---HHHHHHHHHHHHHHHSS-HHHHHHHHHTT-HHHHHHHTT-S-HHHHHHHHHHHHHHHTS-HHHHHHHHHTTHHHHHHHHHT-S-HHHHHHHHHHHHHHHTS-HHHHHHHHHTT-HHHHHHHHHHS-HHHHHHHHHHHHHHHHH--HHHHHHHHHTT-HHHHHHHTTSS-HHHHHHHHHHHHHHHHHHHHHHHHHT----HHHHHHHHTTHHHHHHHHTT-SSHHHHHHHHHHHHHHTS--S--TTTS-EEETTTTEEE-------------'

In [16]:
''.join(test_df.amino_acid)

'MTTPGKENFRLKSYKNKSLNPDEMRRRREEEGLQLRKQKREEQLFKRRNVATAEEETEEEVMSDGGFHEAQISNMEMAPGGVITSDMIEMIFSKSPEQQLSATQKFRKLLSKEPNPPIDEVISTPGVVARFVEFLKRKENCTLQFESAWVLTNIASGNSLQTRIVIQAGAVPIFIELLSSEFEDVQEQAVWALGNIAGDSTMCRDYVLDCNILPPLLQLFSKQNRLTMTRNAVWALSNLCRGKSPPPEFAKVSPCLNVLSWLLFVSDTDVLADACWALSYLSDGPNDKIQAVIDAGVCRRLVELLMHNDYKVVSPALRAVGNIVTGDDIQTQVILNCSALQSLLHLLSSPKESIKKEACWTISNITAGNRAQIQTVIDANIFPALISILQTAEFRTRKEAAWAITNATSGGSAEQIKYLVELGCIKPLCDLLTVMDSKIVQVALNGLENILRLGEQEAKRNGTGINPYCALIEEAYGLDKIEFLQSHENQEIYQKAFDLIEHYFGTEDEDSSIAPQVDLNQQQYIFQQCEAPMEGFQL'

In [11]:
seqID_list = [f.replace('.pkl','').split('/')[1] for f in pkl_file_list]

In [12]:
pred_ss_df = pd.DataFrame()

for seqID, pkl_df in zip(seqID_list,pkl_df_list):
    prediction = ''.join(pkl_df.SS)
    sequence = ''.join(pkl_df.amino_acid)
    pred_ss_df = pd.concat([pred_ss_df, pd.Series({'seqID':int(seqID),'AFpred':prediction,'AA':sequence})],axis=1,sort=False)

In [13]:
pred_ss_df = pred_ss_df.T

In [14]:
pred_ss_df = pred_ss_df.reset_index(drop=True)

In [15]:
pred_ss_df

,seqID,AFpred,AA
0,25673,-----SSHHHHTTSSSS---HHHHHHHHHHHHHHHHHHHHHHHHHH...,MTTPGKENFRLKSYKNKSLNPDEMRRRREEEGLQLRKQKREEQLFK...
1,24810,-HHHHHHHHHHHHHHHHHTTS-------------------------...,MAALKSWLSRSVTSFFRYRQCLCVPVVANFKKRCFSELIRPWHKTV...
2,24706,---------------TTTHHHHHHTS-S--S---------------...,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...
3,25429,------------------------------------------EEEE...,METEQPEETFPNTETNGEFGKRPAEDMEEEQAFKRSRNTDEMVELR...
4,25798,--EEEEEEEE----S-HHHHHHHHHHHHHHHHHTT-SEEE--TTTT...,MSTFRLALIQLQVSSIKSDNLTRACSLVREAAKQGANIVSLPECFN...
...,...,...,...
446,25500,---SBTTB--------------------------------------...,MAHNAGAAAAAGTHSAKSGGSEAALKEGGSAAALSSSSSSSAAAAA...
447,24832,----TTEEEEEEEEEE-BSSSHHHHHHHHHHHTT--EEEEE-SSSE...,MAKNTSCGVQLRIRGKVQGVGFRPFVWQLAQQLNLHGDVCNDGDGV...
448,26150,---TTTT--B--GGGS-HHHHHHHHHHHHHHHHS--TTTTTT-EEE...,MANPLYQKHIISINDLSRDDLNLVLATAAKLKANPQPELLKHKVIA...
449,25199,---HHHHHHHHHHHHHT--TTT--TT-BTTTTS---HHHHHHHHHH...,MSTIEERVKKIIGEQLGVKQEEVTNNASFVEDLGADSLDTVELVMA...


In [16]:
truth_ss_df = pd.read_csv('data/1507_sec.csv',names=['seqID','domain','SS'])

In [17]:
# drop last 10 rows as there are no pdb ids for them
truth_ss_df = truth_ss_df.iloc[:-10,:]
truth_ss_df

,seqID,domain,SS
0,24695,d1a12a_,-------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...
1,24696,d1a1xa_,---------SEEEEEETTEEEETTS-EEEEEEEE-SS-EEEEEE--...
2,24697,d1a62a1,-BHHHHHTS-HHHHHHHHHTTT----TTS-HHHHHHHHHHHHHHTT-
3,24698,d1a9xa1,SSS-SSS--SS-TT-TTHHHHHHHHHHT-BTTHHHHHHHHHHTTB-...
4,24699,d1a9xb1,--EEEEEETTS-EEEEEE-S-SEEEEEEEEEE--SS-HHHHHT-GG...
...,...,...,...
1492,26187,d4lrsa2,-TTHHHHHHHHHHHHTS-HHHHHHHHHHHT--TT-THHHHHHHHHH...
1493,26188,d4lwra_,---TTS--EEEE----STTTTTSGGGHHHHHHHHHHHHHT---EEE...
1494,26189,d4mf5a2,SS-SSHHHHHHHHHHHHHHHHTHHHHHHHHHHHHHT--TT----HH...
1495,26190,d4mjra1,-EEEEEHHHHHHHHHHHT---GGGGEEEEEEETTEEEEEEE-SSEE...


Merge the secondary structure dataframe and the predictions dataframe

In [18]:
merged_df = truth_ss_df.merge(pred_ss_df, how='right',on=['seqID'])

In [19]:
merged_df

,seqID,domain,SS,AFpred,AA
0,24695,d1a12a_,-------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...,-------------S-------------TT-----BEEEEEEE-TTS...,MSPKRIAKRRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVG...
1,24696,d1a1xa_,---------SEEEEEETTEEEETTS-EEEEEEEE-SS-EEEEEE--...,-------S--SEEEEEETTEEEETT--EEEEEEEEETTEEEEEEE-...,MAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQ...
2,24697,d1a62a1,-BHHHHHTS-HHHHHHHHHTTT----TTS-HHHHHHHHHHHHHHTT-,-BHHHHHHS-HHHHHHHHHHTT--SGGGS-HHHHHHHHHHHHHHTT...,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSG...
3,24706,d1aiea_,--------SHHHHHHHHHHHHHHHHHHHH--,---------------TTTHHHHHHTS-S--S---------------...,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...
4,24713,d1b79a_,---HHHHHHHHHHHHHH-GGGHHHHHTT--GGGSSSHHHHHHHHHH...,------------------TTTTTS------HHHHHHHHHHHHH-GG...,MAGNKPFNKQQAEPRERDPQVAGLKVPPHSIEAEQSVLGGLMLDNE...
...,...,...,...,...,...
446,26178,d4j9yb_,--TT-EE----HHHHHHHHHHHHHHHHHHHHHHHHHHS-SS--HHH...,-TT-------------------------------------------...,MSSCRYNGGVMRPLSNLSSSRRNLHEMDSEAQPLQPPASVVGGGGG...
447,26183,d4kefa_,--SS--EE-HHHHHHHHHHHHH---SEEEEEE-TTSSEEEEEEEE-...,------EE-HHHHHHHHHHHTT---SEEEEEE-TTSSEEEEEEEE-...,MSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...
448,26184,d4klia1,-TTHHHHHHHHHHHHHHHHTT--HHHHHHHHHHHHHHHH--S----...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...
449,26185,d4klia2,HHHHHHHHHTTSTT--HHHHHHHHHTT--SHHHHHHTGGGS-HHHH...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...


In [20]:
merged_df.to_csv('451_merged.csv',index=False)

### A slight detour here...
> I realize that the sequences and predicted ss strings are not trimmed down to the scope domain

> I need to get a dictionary of seqID-> (startIdx,endIdx) from the json files

In [1]:
import json
import os
import pandas as pd

In [2]:
os.chdir('/cluster/gjb_lab/2472402')

In [3]:
merged_df = pd.read_csv('451_merged.csv').drop(columns=['AA'])

In [4]:
ref_df = pd.read_csv('1497_url.csv').drop(columns=['url']) # to get pdbid from seqId

In [5]:
all_in_one_df = ref_df.merge(merged_df,how='right',on=['seqID','domain'])

In [7]:
all_in_one_df

,seqID,domain,pdbId,chain,entity,uniprotId,sequence,SS,AFpred
0,24695,d1a12a_,1a12,a,1.0,P18754,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...,-------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...,-------------S-------------TT-----BEEEEEEE-TTS...
1,24696,d1a1xa_,1a1x,a,1.0,P56278,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,---------SEEEEEETTEEEETTS-EEEEEEEE-SS-EEEEEE--...,-------S--SEEEEEETTEEEETT--EEEEEEEEETTEEEEEEE-...
2,24697,d1a62a1,1a62,a,1.0,P0AG30,*NLTELKNTPVSELITLGEN*GLENLAR*RKQDIIFAILKQHAKSG...,-BHHHHHTS-HHHHHHHHHTTT----TTS-HHHHHHHHHHHHHHTT-,-BHHHHHHS-HHHHHHHHHHTT--SGGGS-HHHHHHHHHHHHHHTT...
3,24706,d1aiea_,1aie,a,1.0,P04637,EYFTLQIRGRERFEMFRELNEALELKDAQAG,--------SHHHHHHHHHHHHHHHHHHHH--,---------------TTTHHHHHHTS-S--S---------------...
4,24713,d1b79a_,1b79,a,1.0,P0ACB0,MASHMERDPQVAGLKVPPHSIEAEQSVLGGLMLDNERWDDVAERVV...,---HHHHHHHHHHHHHH-GGGHHHHHTT--GGGSSSHHHHHHHHHH...,------------------TTTTTS------HHHHHHHHHHHHH-GG...
...,...,...,...,...,...,...,...,...,...
446,26178,d4j9yb_,4j9y,b,1.0,P70604,MGRKLELTKAEKHVHNFMMDTQLTKRVKNAAANVLRETWLIYKNTK...,--TT-EE----HHHHHHHHHHHHHHHHHHHHHHHHHHS-SS--HHH...,-TT-------------------------------------------...
447,26183,d4kefa_,4kef,a,1.0,Q03048,GSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,--SS--EE-HHHHHHHHHHHHH---SEEEEEE-TTSSEEEEEEEE-...,------EE-HHHHHHHHHHHTT---SEEEEEE-TTSSEEEEEEEE-...
448,26184,d4klia1,4kli,a,4.0,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,-TTHHHHHHHHHHHHHHHHTT--HHHHHHHHHHHHHHHH--S----...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...
449,26185,d4klia2,4kli,a,4.0,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,HHHHHHHHHTTSTT--HHHHHHHHHTT--SHHHHHHTGGGS-HHHH...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...


In [8]:
# make sure no NaN at all
all_in_one_df.isna().sum()

seqID        0
domain       0
pdbId        0
chain        0
entity       0
uniprotId    0
sequence     0
SS           0
AFpred       0
dtype: int64

In [9]:
all_in_one_df.to_csv('451_all_in_one.csv',index=False)

In [10]:
all_in_one_df = pd.read_csv('451_all_in_one.csv')

In [11]:
for i,row in all_in_one_df.iterrows():
    json_file = 'pdb-json-files/%s.json' % row.pdbId
    if not os.path.exists(json_file):
        print(f'does not exist: {json_file}. continuing...')
    else:
        with open(json_file,'r') as f:
            line = f.read()
            assert line
        # recurse down to the dictionary for that uniprot mapping
        base_dictionary = json.loads(line)
        instance_dictionary = base_dictionary.values().__iter__().__next__()['UniProt'][str(row.uniprotId)]
        assert instance_dictionary
        startIdx = instance_dictionary['mappings'][0]['unp_start']
        endIdx = instance_dictionary['mappings'][0]['unp_end']
        if startIdx < endIdx:
            all_in_one_df.loc[i,'AFpred'] = all_in_one_df.loc[i,'AFpred'][startIdx-1:endIdx]
            all_in_one_df.loc[i,'sequence'] = all_in_one_df.loc[i,'sequence'][startIdx-1:endIdx]
        else:
            # dunno why startIdx greater than end Idx
            print(f'{row.uniprotId} has startIdx {startIdx} greater than {endIdx}')
            pass

all_in_one_df.to_csv('451_all_in_one_trimmed.csv',index=False)

P04637 has startIdx 326 greater than 224
P34707 has startIdx 540 greater than 310
Q14653 has startIdx 196 greater than 113
O75643 has startIdx 1808 greater than 625
Q13576 has startIdx 1476 greater than 1067


In [12]:
all_in_one_df

,seqID,domain,pdbId,chain,entity,uniprotId,sequence,SS,AFpred
0,24695,d1a12a_,1a12,a,1.0,P18754,IPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENVMERKKPAL...,-------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...,-----S-------------TT-----BEEEEEEE-TTSTT-S-TT-...
1,24696,d1a1xa_,1a1x,a,1.0,P56278,SAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQ...,---------SEEEEEETTEEEETTS-EEEEEEEE-SS-EEEEEE--...,------S--SEEEEEETTEEEETT--EEEEEEEEETTEEEEEEE--...
2,24697,d1a62a1,1a62,a,1.0,P0AG30,*NLTELKNTPVSELITLGEN*GLENLAR*RKQDIIFAILKQHAKSG...,-BHHHHHTS-HHHHHHHHHTTT----TTS-HHHHHHHHHHHHHHTT-,-BHHHHHHS-HHHHHHHHHHTT--SGGGS-HHHHHHHHHHHHHHTT...
3,24706,d1aiea_,1aie,a,1.0,P04637,EYFTLQIRGRERFEMFRELNEALELKDAQAG,--------SHHHHHHHHHHHHHHHHHHHH--,---------------TTTHHHHHHTS-S--S---------------...
4,24713,d1b79a_,1b79,a,1.0,P0ACB0,VAGLKVPPHSIEAEQSVLGGLMLDNERWDDVAERVVADDFYTRPHR...,---HHHHHHHHHHHHHH-GGGHHHHHTT--GGGSSSHHHHHHHHHH...,--------TTTTTS------HHHHHHHHHHHHH-GGGHHHHHTT--...
...,...,...,...,...,...,...,...,...,...
446,26178,d4j9yb_,4j9y,b,1.0,P70604,,--TT-EE----HHHHHHHHHHHHHHHHHHHHHHHHHHS-SS--HHH...,HHHS--HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHS-...
447,26183,d4kefa_,4kef,a,1.0,Q03048,SRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETSTD...,--SS--EE-HHHHHHHHHHHHH---SEEEEEE-TTSSEEEEEEEE-...,-----EE-HHHHHHHHHHHTT---SEEEEEE-TTSSEEEEEEEE--...
448,26184,d4klia1,4kli,a,4.0,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,-TTHHHHHHHHHHHHHHHHTT--HHHHHHHHHHHHHHHH--S----...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...
449,26185,d4klia2,4kli,a,4.0,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,HHHHHHHHHTTSTT--HHHHHHHHHTT--SHHHHHHTGGGS-HHHH...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...


In [15]:
err_count = 0
for i,row in all_in_one_df.iterrows():
    ss_length = len(row.SS)
    af_length = len(row.AFpred)
    if ss_length != af_length:
        err_count+=1


In [16]:
err_count

394

I don't know what is going on - are the uniprot start and end positions not referring to the positions on the pdb sequence?